# ReACT Agent

### Overview

When discussing Large Language Model (LLM) Agents, the ReACT architecture is often at the forefront. This architecture is both simple and powerful, making it a popular choice in many mainstream Agentic frameworks. In this tutorial, we'll delve into how ReACT functions internally and showcase how to create a ReACT agent using HybridAGI, a versatile, graph-based Agentic framework.

### Why ReACT?

While the ReACT architecture provides a solid foundation, it is not always the best choice for every situation, particularly for complex tasks. Its simplicity can lead to limitations in control and efficiency. However, for those new to Graph-based Prompt Programming, ReACT offers an excellent starting point. With HybridAGI, you can build a ReACT agent and then expand and refine its capabilities by incorporating additional actions and decision-making processes, all within the same framework. This adaptability is one of the standout features of HybridAGI.

### Key Components of a ReACT Agent

A ReACT agent fundamentally consists of two main elements:

1. **Tool Selection:** Deciding which tool or action to utilize based on the context and requirements.
2. **Iterative Adaptation:** Continuously adjusting the agent's responses in response to user input and interactions.

In the following sections, we'll guide you through building and refining a ReACT agent, helping you grasp the basics and leverage HybridAGI to enhance your agent's functionality.


In [1]:
# Lets Import our Chain of Thought Graph
import hybridagi.core.graph_program as gp

## The Building Blocks
In order to build Our Chain of Though (COT) ReACT agent we need to implement the Decisions our Agent is to make and specify which Actions it's allowed to take
- Decisions: We specify what question the Agent should be asking itself in order to move on
- Actions: We specify a prompt as to what the Agent is trying to accomplish with this Action and what tool it's allowed to use in order to complete that Action

In [2]:
# Initiate our Graph
main = gp.GraphProgram(
    name = 'main',
    description = 'The main program'
)

In [3]:
# Specify the Decision to be made
main.add(gp.Decision(
    id = "tool_choice",
    purpose = "Choose the next tool to use",
    question = \
"""Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish""",
))

In [4]:
#Specify the Actions our Agent can take, the prompt that specifies the goal of the action, and the tool the Agent will use at this step
main.add(gp.Action(
    id = "ask_user",
    purpose = "Ask the user",
    tool = "AskUser",
    prompt = "Ask a question to the user",
))

main.add(gp.Action(
    id = "finish",
    purpose = "End the conversation and give the final answer",
    tool = "Speak",
    prompt = "Please give the final answer, if you don't know just say that you don't know",
))

## Connecting the Pieces
Now that we have specified the "Nodes" in our Chain of Thought we need to connect the Nodes appropriately to impart this "logic chain" on the agent

In [5]:
main.connect("start", "tool_choice")
main.connect("tool_choice", "ask_user", label="Ask User")
main.connect("tool_choice", "finish", label="Finish")
main.connect("ask_user", "tool_choice")
main.connect("finish", "end")

main.build()

print(main)

// @desc: The main program
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(tool_choice:Decision {
  id: "tool_choice",
  purpose: "Choose the next tool to use",
  question: "Which tool to use for the next step?\nUse the context to help you choose.\nTo give the final answer just select finish"
}),
(ask_user:Action {
  id: "ask_user",
  purpose: "Ask the user",
  tool: "AskUser",
  prompt: "Ask a question to the user"
}),
(finish:Action {
  id: "finish",
  purpose: "End the conversation and give the final answer",
  tool: "Speak",
  prompt: "Please give the final answer, if you don't know just say that you don't know"
}),
// Structure declaration
(start)-[:NEXT]->(tool_choice),
(tool_choice)-[:ASK_USER]->(ask_user),
(tool_choice)-[:FINISH]->(finish),
(ask_user)-[:NEXT]->(tool_choice),
(finish)-[:NEXT]->(end)


## Building our Agent
To set up our Agent we need to:
1. Decide on an LLM to use
2. Upload our Chain of Thought Graph into the Agent's Program Memory
3. Declare the Agent State (this allows the Agent to orient itself inside the Graph)
4. Declare our Tools
5. Instantiate the Agent itself

### 1. Decide on the LLM to use
HybridAGI interacts over the top of the dspy library, in this tutorial I will be runnning the Agent Locally over Ollama
- To Download Ollama you can do so simply [here](https://ollama.com/download)
- Then you can either launch Ollama as from the App or in your terminal by running the command:


  ```bash
  ollama serve
  ollama run <your-model>
  ```

- To verify Ollama is running you can go to http://localhost:11434/ for the "Ollama is running" notification

#### Ollama Model Options (I will be using Mistral for this Tutorial)

<img src="../img/ollama_models.png" alt="Placeholder Image" width="40%">

#### In Google Colab
If you're following this notebook in Google Colab you can get Ollama's server running by following these steps:
- Installing the Terminal Package
```python
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm


- Opening the Terminal
```bash
%xterm

- Inside the Terminal Run this Linux Command:

```bash
curl -fsSL https://ollama.com/install.sh | sh

- Finally Serve your Model
```bash
ollama serve
ollama run <your-model>

In [6]:
import dspy
lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n", "\n---"])
dspy.configure(lm=lm)

### 2. Upload the COT to our Program's Memory

In [7]:
from hybridagi.memory.integration.local import LocalProgramMemory
program_memory = LocalProgramMemory(index_name="react_agent")
program_memory.update(main)

### 3. Orient our Agent in the Graph using Agent State

In [8]:
from hybridagi.core.datatypes import AgentState
agent_state = AgentState()

### 4. Declare our Tools

In [9]:
from hybridagi.modules.agents.tools import (
    SpeakTool,
    AskUserTool
)
tools = [
    SpeakTool(
        agent_state = agent_state,
    ),
    AskUserTool(
        agent_state = agent_state, simulated=True
    )
]

### 5. Build the Agent

In [10]:
from hybridagi.modules.agents import GraphInterpreterAgent
agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
)

## Now Lets Talk to Our ReACT Agent!

In [11]:
from hybridagi.core.datatypes import Query
result = agent(Query(query="Please teach me how to how to bake a chocolate, vanilla or strawberry cake"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: Please teach me how to how to bake a chocolate, vanilla or strawberry cake
--- Step 1 ---
Decision Purpose: Choose the next tool to use
Decision Question: Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish
Decision: FINISH
--- Step 2 ---
Action Purpose: End the conversation and give the final answer
Action: {
  "message": "To bake a chocolate, vanilla or strawberry cake, follow these steps:\n\n1. Gather your ingredients: flour, sugar, baking powder, salt, eggs, milk, and butter for the base. For flavoring, choose cocoa powder for chocolate cake, vanilla extract for vanilla cake, or strawberries for strawberry cake.\n\n2. Preheat your oven to 350\u00b0F (175\u00b0C). Grease and flour a cake pan.\n\n3. In a large bowl, combine the dry ingredients: flour, sugar, baking powder, and salt.\n\n4. In another bowl, beat the eggs and gradually add milk and melted butter. If you're

### Analyzing Response
It looks like our Agent felt confident enough to proceed directly to an answer, but what if we force its hand to ask a question?
- *Note: Our AskUserTool is set to simulation=True, meaning the AskUser answer will be simulated by the LLM rather than taking actual input of a user*

In [12]:
result = agent(Query(query="Please teach me how to how to bake a"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: Please teach me how to how to bake a
--- Step 1 ---
Decision Purpose: Choose the next tool to use
Decision Question: Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish
Decision: ASK_USER
--- Step 2 ---
Action Purpose: Ask the user
Action: {
  "question": "What baking recipe or type of baked good would you like me to teach you how to make?",
  "answer": "A simple chocolate chip cookie recipe, please."
}
--- Step 3 ---
Decision Purpose: Choose the next tool to use
Decision Question: Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish
Decision: ASK_USER
--- Step 4 ---
Action Purpose: Ask the user
Action: {
  "question": "What ingredients do you have for making a simple chocolate chip cookie recipe?",
  "answer": "I have flour, sugar, eggs, baking soda, and chocolate chips."
}
--- Step 5 ---
Decision Purpose: C

## The Customizable nature of HybridAGI
Wait a second. After the agent asked the user for clarification on what should be baked it should have changed the Objective of the conversation to teaching the user how to make the Chocolate Chip Cookies. Instead, the agent chose to ask another question: what ingredients the user had access to. 
To closer align our Agent to our intended purpose we can show off the customizable nature of HybridAGI by changing our decision process and introducting the UpdateObjective tool!

### Creating our New Building Blocks

In [13]:
main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Decision(
    id = "tool_choice",
    purpose = "Choose the next tool to use",
    question = \
"""Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish""",
))

main.add(gp.Action(
    id = "ask_user",
    purpose = "Ask the user",
    tool = "AskUser",
    prompt = "Ask a question to the user",
))

main.add(gp.Action(
    id='refine_objective',
    purpose='Refine the objective',
    tool='UpdateObjective',
    prompt='Please refine the User Objective'
))

main.add(gp.Action(
    id = "finish",
    purpose = "End the conversation and give the final answer",
    tool = "Speak",
    prompt = "Please give the final answer, if you don't know just say that you don't know",
))

main.connect("start", "tool_choice")
main.connect("tool_choice", "ask_user", label="Ask User")
main.connect("tool_choice", "finish", label="Finish")
main.connect("ask_user", "refine_objective")
main.connect('refine_objective', 'finish')
main.connect("finish", "end")

main.build()

print(main)

// @desc: The main program
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(tool_choice:Decision {
  id: "tool_choice",
  purpose: "Choose the next tool to use",
  question: "Which tool to use for the next step?\nUse the context to help you choose.\nTo give the final answer just select finish"
}),
(ask_user:Action {
  id: "ask_user",
  purpose: "Ask the user",
  tool: "AskUser",
  prompt: "Ask a question to the user"
}),
(refine_objective:Action {
  id: "refine_objective",
  purpose: "Refine the objective",
  tool: "UpdateObjective",
  prompt: "Please refine the User Objective"
}),
(finish:Action {
  id: "finish",
  purpose: "End the conversation and give the final answer",
  tool: "Speak",
  prompt: "Please give the final answer, if you don't know just say that you don't know"
}),
// Structure declaration
(start)-[:NEXT]->(tool_choice),
(tool_choice)-[:ASK_USER]->(ask_user),
(tool_choice)-[:FINISH]->(finish),
(ask_user)-[:NEXT]->(refine_objective)

In [15]:
from hybridagi.modules.agents.tools import UpdateObjectiveTool

# Upload our Graph into memory
program_memory = LocalProgramMemory(index_name="react_agent")
program_memory.update(main)

# Orient our Agent
agent_state = AgentState()

# Set up our Tools
tools = [
    SpeakTool(
        agent_state = agent_state,
    ),
    AskUserTool(
        agent_state = agent_state, simulated=True
    ),
    UpdateObjectiveTool(agent_state=agent_state)
]

# Build our Agent
agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
)

In [16]:
result = agent(Query(query="Please teach me how to how to bake a"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: Please teach me how to how to bake a
--- Step 1 ---
Decision Purpose: Choose the next tool to use
Decision Question: Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish
Decision: ASK_USER
--- Step 2 ---
Action Purpose: Ask the user
Action: {
  "question": "What baking recipe or type of baked good would you like me to teach you how to make?",
  "answer": "A simple chocolate chip cookie recipe, please."
}
--- Step 3 ---
Action Purpose: Refine the objective
Action: {
  "new_objective": "Teach me a simple chocolate chip cookie recipe.\n\nContext: The user has requested to learn how to bake a simple chocolate chip cookie recipe.\n\nPurpose: Provide instructions for baking a simple chocolate chip cookie recipe.\n\nPrompt: Begin by explaining the ingredients and steps required to make a simple chocolate chip cookie recipe."
}
--- Step 4 ---
Action Purpose: End the conversation a

### Saving our COT Graph
Perfect! Now our Interactive ReACT Agent acts exactly as expected. Now to take this model with us to our desired application or to save for use later we can save our Chain of Thought Graph with the .save() command

In [ ]:
main.save()

#### The saved graph can then be uploaded directly into program memory like this:

In [ ]:
from hybridagi.readers import GraphProgramReader

#Set up the reader for our .cypher file
reader = GraphProgramReader()
main = reader('main.cypher')